In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x_feature = tf.feature_column.numeric_column('f1')

In [3]:
train_features = np.array([1., 2., 3., 4.])
train_labels = np.array([1.5, 3.5, 5.5, 7.5])

def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices(({"f1": train_features}, train_labels))
    dataset = dataset.shuffle(1000).repeat().batch(2)
    #return dataset.make_one_shot_iterator().get_next()
    return dataset

test_features = np.array([5., 6., 7.])
test_labels = np.array([9.5, 11.5, 13.5])

def test_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices(({"f1": test_features}, test_labels))
    dataset = dataset.repeat(1).batch(len(test_features)) # batch cannot be none
    return dataset

samples = np.array([8., 9.])
def predict_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices(({"f1": samples}))
    dataset = dataset.repeat(1).batch(len(samples)) # batch cannot be none
    return dataset

In [4]:
regressor = tf.estimator.LinearRegressor(
    feature_columns=[x_feature],
    model_dir='./output'
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000004FA06A0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
regressor.train(input_fn=train_input_fn, steps=2500) # incremental
#regressor.train(input_fn=train_input_fn, max_steps=2500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./output\model.ckpt.
INFO:tensorflow:loss = 32.5, step = 1
INFO:tensorflow:global_step/sec: 1052.63
INFO:tensorflow:loss = 0.214375, step = 101 (0.096 sec)
INFO:tensorflow:global_step/sec: 1351.35
INFO:tensorflow:loss = 0.1400778, step = 201 (0.075 sec)
INFO:tensorflow:global_step/sec: 1190.47
INFO:tensorflow:loss = 0.16759762, step = 301 (0.083 sec)
INFO:tensorflow:global_step/sec: 1190.48
INFO:tensorflow:loss = 0.16801906, step = 401 (0.085 sec)
INFO:tensorflow:global_step/sec: 1204.82
INFO:tensorflow:loss = 0.25878322, step = 501 (0.082 sec)
INFO:tensorflow:global_step/sec: 1190.48
INFO:tensorflow:loss = 0.027490944, step = 601 (0.084 sec)
INFO:tensorflow:global_step/sec: 1075.27
INFO:tensorflow:loss = 0.

In [6]:
average_loss = regressor.evaluate(input_fn=test_input_fn)["average_loss"]
print(f"Average loss in testing: {average_loss:.4f}")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-12-07:14:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./output\model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-12-07:14:06
INFO:tensorflow:Saving dict for global step 2500: average_loss = 0.00020292633, global_step = 2500, label/mean = 11.5, loss = 0.00060877897, prediction/mean = 11.486214
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2500: ./output\model.ckpt-2500
Average loss in testing: 0.0002


In [7]:
predictions = list(regressor.predict(input_fn=predict_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./output\model.ckpt-2500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [8]:
for input, p in zip(samples, predictions):
    v  = p["predictions"][0]
    print(f"{input} -> {v:.4f}")

8.0 -> 15.4774
9.0 -> 17.4730
